In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

1. Get batch from the training set
2. Pass batch to Network
3. Calculate the loss (difference between the predicted values and the true values)
4. Calculate the gradient of the loss function w.r.t. the network's weights
5. Update the weights using the gradients to reduce the loss
6. Repeat 1-5 until one epoch (Complete pass through all samples of the training set) is completed
7. Repeat steps 1-6 for as many epochs required to obtain the desired level of accuracy

In [86]:
print(torch.__version__)
print(torchvision.__version__)

1.3.0
0.4.1a0+d94043a


In [87]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [88]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [89]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t) 
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)

        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)

        return t

In [90]:
network = Network()

In [91]:
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=100
)
batch = next(iter(train_loader))
images, labels = batch

###Calculating Loss###

In [92]:
preds = network(images)
loss = F.cross_entropy(preds, labels) # Calculating the loss
loss.item()

2.309378147125244

Calculating the Gradients

In [93]:
print(network.conv1.weight.grad)

None


In [94]:
loss.backward() # Caluclating the gradients

In [95]:
network.conv1.weight.grad.shape

torch.Size([6, 1, 5, 5])

Update the Weights

In [96]:
optimizer = optim.Adam(network.parameters(), lr=0.01)
# or SGD

In [97]:
loss.item()

2.309378147125244

In [98]:
get_num_correct(preds, labels)

9

In [99]:
optimizer.step() # Updating the weights

In [100]:
preds = network(images)
loss = F.cross_entropy(preds, labels)

In [101]:
loss.item()

2.283230781555176

In [102]:
get_num_correct(preds, labels)

19